[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanhuguet/intro_to_nlp/blob/main/notebooks/06-text-clustering-with-embeddings.ipynb)

In [ ]:
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    
try:
    from datasets import load_dataset
except:
    install("datasets")
    from datasets import load_dataset

In [ ]:
import pandas as pd

### Load the dataset

In [ ]:
dataset = load_dataset("yelp_review_full")

### Convert it to pandas a random sample of 1000 articles

In [ ]:
df = pd.DataFrame(dataset["train"].shuffle(42).select(range(...)))

## Embedd sentences

In [ ]:
try:
    from sentence_transformers import SentenceTransformer
except:
    install("sentence-transformers")
    from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("distiluse-base-multilingual-cased-v2", device="...")

In [ ]:
sentence_embedding = model.encode("...")

In [ ]:
sentence_embedding.shape

## Get sentence embeddings for all texts

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
## use the encode method of sentence transformers
df["embedding"] = df["text"].progress_apply()

## Let's reduce the dimensionality and visualize the data in 2d

In [ ]:
try:
    import umap
except:
    install("umap-learn")
    import umap

In [ ]:
# reduce it to 2 dimensions

def reduce_dimensions(embeddings,
                      n_components=..,
                      n_neighbors=8,
                      random_state = 42):
    
    umap_embeddings = (umap.UMAP(n_neighbors=n_neighbors, 
                                n_components=n_components, 
                                metric='cosine',
                                 n_epochs=10,
                                random_state=random_state)
                            .fit_transform(embeddings))
    df = pd.DataFrame(umap_embeddings, index=embeddings.index)
    return df

In [ ]:
#explode the embeddings
embeddings = df["embedding"].apply(pd.Series)

In [ ]:
embeddings_2d = reduce_dimensions(embeddings)

In [ ]:
embeddings_2d

### Let's visualize them...

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(embeddings_2d, x=0, y=1)

Nice, let's see if we can cluster them....

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
clst = AgglomerativeClustering(n_clusters=10)

In [ ]:
clusters = clst.fit_predict(embeddings)

## Let's assign the clusters to the visualization

In [ ]:
sns.scatterplot(embeddings_2d, x=0, y=1, c=clusters)

## Now, let's review the contents of the clusters...

In [ ]:
df["cluster"] = clusters

In [ ]:
df.sort_values(by="cluster")

In [ ]:
df.loc[df["cluster"] == 9, "text"]